In [1]:
import pandas as pd
import requests
from time import sleep
from requests_html import HTML

In [2]:
base_url = 'https://stackoverflow.com/questions/tagged/'
tag = 'python'
query_filter = 'Votes'
url = f"{base_url}{tag}?tab={query_filter}"
url

'https://stackoverflow.com/questions/tagged/python?tab=Votes'

In [3]:
r = requests.get(url)
html_txt = r.text

In [4]:
html = HTML(html=html_txt)

In [5]:
question_summaries = html.find('.question-summary')
print(question_summaries)

[<Element 'div' class=('question-summary',) id='question-summary-231767'>, <Element 'div' class=('question-summary',) id='question-summary-419163'>, <Element 'div' class=('question-summary',) id='question-summary-394809'>, <Element 'div' class=('question-summary',) id='question-summary-100003'>, <Element 'div' class=('question-summary',) id='question-summary-82831'>, <Element 'div' class=('question-summary',) id='question-summary-38987'>, <Element 'div' class=('question-summary',) id='question-summary-89228'>, <Element 'div' class=('question-summary',) id='question-summary-273192'>, <Element 'div' class=('question-summary',) id='question-summary-522563'>, <Element 'div' class=('question-summary',) id='question-summary-952914'>, <Element 'div' class=('question-summary',) id='question-summary-136097'>, <Element 'div' class=('question-summary',) id='question-summary-509211'>, <Element 'div' class=('question-summary',) id='question-summary-3437059'>, <Element 'div' class=('question-summary

In [6]:
print(question_summaries[0].text)

11024
votes
38answers
2.5m views
What does the “yield” keyword do?
What is the use of the yield keyword in Python, and what does it do? For example, I'm trying to understand this code1: def _get_child_candidates(self, distance, min_dist, max_dist): if self....
python iterator generator yield coroutine
asked Oct 23 '08 at 22:21
Alex. S.
125k1515 gold badges5050 silver badges6161 bronze badges


In [7]:
columns = ['votes', 'vote_title', 'num_answers', 'views', 'question', 'short_desc', 'tags', 'date', 'user', 'user_details']

In [8]:
# this_row = list(question_summaries[0].text.split('\n'))
# print(this_row)
# row_data = dict(zip(columns, this_row))
# row_data

In [9]:
key_names = ['question', 'votes']
classes_needed = ['.question-hyperlink', '.vote-count-post']
this_question_el = question_summaries[0]
this_question_el.find('.question-hyperlink', first=True).text

'What does the “yield” keyword do?'

In [10]:
data = []

for q_el in question_summaries:
    questio_data = {}
    for i, _class in enumerate(classes_needed):
        sub_el = q_el.find(_class, first=True)
        keyname = key_names[i]
        questio_data[keyname] = sub_el.text
    data.append(questio_data)
data[0]

{'question': 'What does the “yield” keyword do?', 'votes': '11024'}

In [11]:
def parse_tagged_page(html):
    question_summaries = html.find('.question-summary')
    key_names = ['question', 'votes', 'tags']
    classes_needed = ['.question-hyperlink', '.vote-count-post', '.tags']
    data = []

    for q_el in question_summaries:
        questio_data = {}
        for i, _class in enumerate(classes_needed):
            sub_el = q_el.find(_class, first=True)
            keyname = key_names[i]
            questio_data[keyname] = sub_el.text
        data.append(questio_data)
    return data

In [12]:
def extract_page_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200, 300):
        return []
    html_txt = r.text
    html = HTML(html=html_txt)
    data = parse_tagged_page(html)
    return data

In [13]:
def scrape_tag(tag='python', query_filter='Votes', max_pages=10, pagesize=15):
    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for page in range(max_pages):
        num_page = page + 1
        url = f"{base_url}{tag}?tab={query_filter}&page={num_page}&pagesize={pagesize}"
        datas += extract_page_from_url(url)
        sleep(1)
    return datas

In [14]:
data = scrape_tag()
df = pd.DataFrame(data)

In [15]:
df.head()

,question,votes,tags
0,What does the “yield” keyword do?,11024,python iterator generator yield coroutine
1,What does if __name__ == “__main__”: do?,6701,python namespaces main python-module idioms
2,Does Python have a ternary conditional operator?,6558,python operators ternary-operator conditional-...
3,What are metaclasses in Python?,6099,python oop metaclass python-class python-datam...
4,How do I check whether a file exists without e...,6065,python file file-exists


In [16]:
df.shape

(500, 3)

In [17]:
df.to_csv('stackoverflow.csv', index=False)